# Food Intake 

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db

## Conjunto de tabelas para consultas básicas

In [2]:
DROP TABLE IF EXISTS Intake;

CREATE TABLE Intake (
  SeqN INTEGER NOT NULL,
  DayCode TINYINT NOT NULL,
  DraBF TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Intake DECIMAL(13,7),
  Intake_BW DECIMAL(13,10),
  PRIMARY KEY(SeqN, DayCode, FCID_Code, Cooked_Status, Food_Form, Cooking_Method)
) AS SELECT
  SEQN, DAYCODE, DRABF, FCID_Code, Cooked_Status, Food_Form, Cooking_Method, Intake,Intake_BW
FROM CSVREAD('../../data/food-intake/intake/Commodity_CSFFM_Intake_0510.csv');

In [3]:
DROP TABLE IF EXISTS Recipes;

CREATE TABLE Recipes (
  Food_Code VARCHAR(8),
  Mod_Code VARCHAR(8),
  Ingredient_Num TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Commodity_Weight DECIMAL(5, 2),
  CSFII_9498_IND TINYINT,
  WWEIA_9904_IND TINYINT,
  WWEIA_0510_IND TINYINT,
  PRIMARY KEY(Food_Code, Mod_Code, Ingredient_Num)
) AS SELECT
  Food_Code, Mod_Code, Ingredient_Num, FCID_Code, Cooked_Status, Food_Form, Cooking_Method,
  Commodity_Weight, CSFII_9498_IND, WWEIA_9904_IND, WWEIA_0510_IND
FROM CSVREAD('../../data/food-intake/recipe/Recipes_WWEIA_FCID_0510.csv');

In [4]:
DROP TABLE IF EXISTS FCID_Description;

CREATE TABLE FCID_Description (
  CGN VARCHAR(2),
  CG_Subgroup VARCHAR(6),
  FCID_Code VARCHAR(10),
  FCID_Desc VARCHAR(55),
  PRIMARY KEY (FCID_Code)
) AS SELECT
  cgn, CG_Subgroup, FCID_Code, FCID_Desc
FROM CSVREAD('../../data/food-intake/basics/FCID_Code_Description.csv');

# SELECT Básico

### Todos os Táxis com todos os atributos

In [19]:
SELECT * FROM Intake LIMIT 10;

In [56]:
SELECT * FROM Recipes LIMIT 10;

In [5]:
SELECT * FROM Recipes WHERE Food_Code = '27111300' AND Mod_Code = 0;

In [6]:
SELECT SUM(Commodity_Weight) FROM Recipes WHERE Food_Code = '27111300' AND Mod_Code = 0;

99.79

In [7]:
SELECT MAX(SeqN) FROM Intake;

62160

In [8]:
SELECT COUNT(DISTINCT Food_Code) FROM Recipes;

7154

In [62]:
DROP VIEW IF EXISTS Participation_Profile;

CREATE VIEW Participation_Profile AS
SELECT R.FCID_Code, COUNT(DISTINCT (R.Food_Code, R.Mod_Code)) Participation
FROM Recipes R
GROUP BY R.FCID_Code;

SELECT * FROM Participation_Profile LIMIT 10;

In [34]:
SELECT COUNT(*) FROM Intake;

3499408

In [67]:
DROP VIEW IF EXISTS Commodity_Profile;

CREATE VIEW Commodity_Profile
AS SELECT I.FCID_Code, F.FCID_Desc, F.CGN, F.CG_Subgroup, COUNT(DISTINCT I.SeqN) Popularity,
          SUM(I.Intake) Intake_Sum, AVG(I.Intake) Intake_AVG,
          AVG(I.Intake_BW) Intake_BW_AVG, P.Participation Recipes
FROM Intake I, FCID_Description F, Participation_Profile P
WHERE I.FCID_Code = F.FCID_Code AND I.FCID_Code = P.FCID_Code
GROUP BY I.FCID_Code;

In [46]:
SELECT SUM(Consumption) FROM Commodity_Profile;

1891731

In [68]:
CALL CSVWRITE('../../data/food-intake/computed/commodity-profile.csv',
              'SELECT * FROM Commodity_Profile');

464